In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

In [0]:
#configs = {
#  "fs.azure.account.auth.type": "CustomAccessToken",
#  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
#}

# Optionally, you can add <directory-name> to the source URI of your mount point.
#dbutils.fs.mount(
#  source = "abfss://bronze-parquet-files@weiridatalake.dfs.core.windows.net/",
#  mount_point = "/mnt/bronze",
#  extra_configs = configs)


In [0]:
dbutils.fs.ls("/mnt/bronze")

[FileInfo(path='dbfs:/mnt/bronze/customers.parquet', name='customers.parquet', size=14877, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/employees/', name='employees/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/employees.parquet', name='employees.parquet', size=2711, modificationTime=1723753600000),
 FileInfo(path='dbfs:/mnt/bronze/offices.parquet', name='offices.parquet', size=2362, modificationTime=1723753600000),
 FileInfo(path='dbfs:/mnt/bronze/orderdetails/', name='orderdetails/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/orderdetails.parquet', name='orderdetails.parquet', size=41567, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/orders.parquet', name='orders.parquet', size=9837, modificationTime=1723753599000),
 FileInfo(path='dbfs:/mnt/bronze/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(pat

In [0]:
df_user = spark.read.parquet("/mnt/bronze/customers.parquet")
df_orders = spark.read.parquet("/mnt/bronze/orders.parquet")
df_products = spark.read.parquet("/mnt/bronze/products.parquet")
df_payments = spark.read.parquet("/mnt/bronze/payments.parquet")
df_orderdetails = spark.read.parquet("/mnt/bronze/orderdetails.parquet")
df_employees = spark.read.parquet("/mnt/bronze/employees.parquet")
df_offices = spark.read.parquet("/mnt/bronze/offices.parquet")



In [0]:
write_delta_mode = "append"

# Processing zone location
delta_processing_store_zone = "/mnt/bronze"

# WRITE INTO DELTA FORMAT

df_user.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/user/")
df_orders.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/orders/")
df_products.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/products/")
df_payments.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/payments/")
df_orderdetails.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/orderdetails/")
df_employees.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/employees/")
df_offices.write.mode(write_delta_mode).format("delta").save(delta_processing_store_zone + "/offices/")


In [0]:
%sql

CREATE DATABASE IF NOT EXISTS case_rpn_database_delta

In [0]:
%sql

USE case_rpn_database_delta;

DROP TABLE IF EXISTS customers_bronze;
DROP TABLE IF EXISTS orders_bronze;
DROP TABLE IF EXISTS products_bronze;
DROP TABLE IF EXISTS payments_bronze;
DROP TABLE IF EXISTS order_details_bronze;
DROP TABLE IF EXISTS employees_bronze;
DROP TABLE IF EXISTS offices_bronze;

CREATE TABLE customers_bronze AS 
SELECT * FROM delta.`/mnt/bronze/user/`;

CREATE TABLE orders_bronze AS 
SELECT * FROM delta.`/mnt/bronze/orders/`;

CREATE TABLE products_bronze AS 
SELECT * FROM delta.`/mnt/bronze/products/`;

CREATE TABLE payments_bronze AS 
SELECT * FROM delta.`/mnt/bronze/payments/`;

CREATE TABLE order_details_bronze AS 
SELECT * FROM delta.`/mnt/bronze/orderdetails/`;

CREATE TABLE employees_bronze AS 
SELECT * FROM delta.`/mnt/bronze/employees/`;

CREATE TABLE offices_bronze AS 
SELECT * FROM delta.`/mnt/bronze/offices/`;

num_affected_rows,num_inserted_rows


In [0]:
%sql

USE case_rpn_database_delta;

select c.country, 
       count(case when o.status = 'Cancelled' then 1 else null end) as qtd_cancelled
from customers_bronze c
left join orders_bronze o on c.customer_number = o.customer_number
group by c.country
order by qtd_cancelled desc


country,qtd_cancelled
New Zealand,144
Sweden,72
Spain,72
USA,72
UK,72
Russia,0
Philippines,0
Singapore,0
Germany,0
France,0


In [0]:
%sql
SELECT 
    e.last_name, 
    e.first_name, 
    REGEXP_REPLACE(e.email, '^[^@]+', '****') AS email,
    e.job_title,
    o.country
FROM employees_bronze e 
LEFT JOIN offices_bronze o ON e.office_code = o.office_code 
WHERE e.job_title = 'Sales Rep' 
  AND o.country = 'Japan'


last_name,first_name,email,job_title,country
Nishi,Mami,****@classicmodelcars.com,Sales Rep,Japan
Kato,Yoshimi,****@classicmodelcars.com,Sales Rep,Japan
Nishi,Mami,****@classicmodelcars.com,Sales Rep,Japan
Kato,Yoshimi,****@classicmodelcars.com,Sales Rep,Japan


In [0]:
%sql

USE case_rpn_database_delta;

SELECT 
    p2.product_line, 
    ROUND(SUM(p.amount * o2.quantity_ordered), 2) AS total_amount
FROM customers_bronze c
LEFT JOIN orders_bronze o ON c.customer_number = o.customer_number
LEFT JOIN payments_bronze p ON c.customer_number = p.customer_number
LEFT JOIN order_details_bronze o2 ON o2.order_number = o.order_number 
LEFT JOIN products_bronze p2 ON p2.product_code = o2.product_code 
WHERE p2.product_line IS NOT NULL 
  AND o.status = 'Shipped' 
  AND YEAR(o.order_date) = 2005
GROUP BY p2.product_line


product_line,total_amount
Motorcycles,5458582574645.76
Vintage Cars,10833488306599.68
Ships,2032377274506.24
Trucks and Buses,11007172838246.40
Classic Cars,27079614006779.52
Trains,2265794873159.04
Planes,2139757406864.64


In [0]:
%sql

USE case_rpn_database_delta;


SELECT 
    e.last_name, 
    e.first_name, 
    REGEXP_REPLACE(e.email, '^[^@]+', '****') AS email,
    e.job_title,
    o.country
FROM employees_bronze e 
LEFT JOIN offices_bronze o ON e.office_code = o.office_code 
WHERE e.job_title = 'Sales Rep' 
  AND o.country = 'Japan'


last_name,first_name,email,job_title,country
Nishi,Mami,****@classicmodelcars.com,Sales Rep,Japan
Kato,Yoshimi,****@classicmodelcars.com,Sales Rep,Japan
Nishi,Mami,****@classicmodelcars.com,Sales Rep,Japan
Kato,Yoshimi,****@classicmodelcars.com,Sales Rep,Japan
